# **Práctica: Flujos de Datos en Clasificación, Concept Drift, Agrupamiento, Anomalías, Texto y Ensembles**


## **Carga de Datos**

Cargamos el dataset que contiene los SMS con sus características extraídas y una columna que indica si es SPAM o no.

In [9]:
import pandas as pd

# Cargar los datos
df = pd.read_csv('spam_SMS_ampliado.csv')
df.columns

Index(['spam', 'text', 'num_caracteres', 'num_palabras', 'num_alfabeticos',
       'num_numericos', 'num_no_alfanum', 'num_divisas', 'num_mayusculas',
       'num_exclamaciones', 'num_interrogaciones', 'num_urls'],
      dtype='object')

In [10]:
df.head(5)

,spam,text,num_caracteres,num_palabras,num_alfabeticos,num_numericos,num_no_alfanum,num_divisas,num_mayusculas,num_exclamaciones,num_interrogaciones,num_urls
0,0,"Go until jurong point, crazy.. Available only ...",111,20,83,0,28,0,3,0,0,0
1,0,Ok lar... Joking wif u oni...,29,6,18,0,11,0,2,0,0,0
2,1,Free entry in 2 a wkly comp to win FA Cup fina...,155,28,97,25,33,0,10,0,0,0
3,0,U dun say so early hor... U c already then say...,49,11,33,0,16,0,2,0,0,0
4,0,"Nah I don't think he goes to usf, he lives aro...",61,13,47,0,14,0,2,0,0,0


In [11]:
len(df)

5574

## **Clasificación**

Vamos a entrenar y evaluar varios clasificadores online (en flujo de datos):
- Árbol Hoeffding (`HoeffdingTreeClassifier`).
- Árbol Adaptativo Hoeffding (`HoeffdingAdaptiveTreeClassifier`).
- Árbol de Decisión Extremadamente Rápido (`ExtremelyFastDecisionTreeClassifier`).

Evaluaremos cada uno utilizando la métrica F1.

In [12]:
from river import tree, metrics

model_standard = tree.HoeffdingTreeClassifier()

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)
    model_standard.learn_one(x, y)
    metric.update(y, y_pred)
    
print(f'F1 Score para HoeffdingTreeClassifier: {metric}')

F1 Score para HoeffdingTreeClassifier: F1: 86.65%


In [13]:
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)
    model_adaptive.learn_one(x, y)
    metric.update(y, y_pred)
    
print(f'F1 Score para HoeffdingAdaptiveTreeClassifier: {metric}')

F1 Score para HoeffdingAdaptiveTreeClassifier: F1: 87.47%


In [14]:
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)

metric = metrics.F1()

for index, row in df.iterrows():
    x = {
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)
    model_extreme.learn_one(x, y)
    metric.update(y, y_pred)

print(f'F1 Score para ExtremelyFastDecisionTreeClassifier: {metric}')

F1 Score para ExtremelyFastDecisionTreeClassifier: F1: 85.23%


## **Detección de Concept Drift**

Para la detección de cambios en el concepto (concept drift), emplearemos varios detectores:
- **KSWIN (`KSWIN`)**: Detector de cambios basado en ventanas deslizantes.
- **EDDM (`EDDM`)**: Monitor de desviación por error esperado.
- **DDM (`DDM`)**: Monitor de desviación por error de detección.

Vamos a implementar estos detectores sobre tres modelos distintos:
1. HoeffdingTreeClassifier
2. HoeffdingAdaptiveTreeClassifier
3. ExtremelyFastDecisionTreeClassifier

### **Concept Drift en HoeffdingTreeClassifier**

Entrenaremos un modelo estándar de árbol Hoeffding y usaremos los detectores de concept drift para evaluar si se detectan cambios en los datos durante el flujo.

In [15]:
from river import tree, metrics, drift
import river
print(river.__version__)

model_standard = tree.HoeffdingTreeClassifier()

metric = metrics.F1()

kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_standard.predict_one(x)
    model_standard.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (HoeffdingTreeClassifier): {metric}')

0.13.0
EDDM - Model drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Model drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Model drift detectado en el ejemplo 174 (F1: 53.57%)
EDDM - Model drift detectado en el ejemplo 224 (F1: 52.46%)
EDDM - Model drift detectado en el ejemplo 297 (F1: 59.26%)
EDDM - Model drift detectado en el ejemplo 389 (F1: 68.47%)
DDM - Model drift detectado en el ejemplo 440 (F1: 71.43%)
DDM - Model drift detectado en el ejemplo 471 (F1: 72.73%)
EDDM - Model drift detectado en el ejemplo 478 (F1: 73.13%)
DDM - Model drift detectado en el ejemplo 502 (F1: 73.91%)
EDDM - Model drift detectado en el ejemplo 557 (F1: 75.64%)


c:\Users\alvar\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: RuntimeWarning: ks_2samp: Exact calculation unsuccessful. Switching to method=asymp.
  res = hypotest_fun_out(*samples, **kwds)


KSWIN - Data drift detectado en el ejemplo 2468 (F1: 86.13%)
KSWIN - Data drift detectado en el ejemplo 3311 (F1: 85.87%)
Resultado final (HoeffdingTreeClassifier): F1: 86.65%


### **Concept Drift en HoeffdingAdaptiveTreeClassifier**

Ahora, vamos a implementar el árbol adaptativo Hoeffding, que puede adaptarse a los cambios detectados en el concepto de los datos, y lo evaluamos con los detectores de drift.

In [17]:
# crear, entrenar y evaluar modelo
model_adaptive = tree.HoeffdingAdaptiveTreeClassifier()

metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_adaptive.predict_one(x)
    model_adaptive.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (HoeffdingAdaptiveTreeClassifier): {metric}')

EDDM - Model drift detectado en el ejemplo 45 (F1: 15.38%)
EDDM - Model drift detectado en el ejemplo 186 (F1: 60.00%)
EDDM - Model drift detectado en el ejemplo 374 (F1: 71.03%)
DDM - Model drift detectado en el ejemplo 376 (F1: 71.56%)
DDM - Model drift detectado en el ejemplo 407 (F1: 73.04%)
KSWIN - Data drift detectado en el ejemplo 2468 (F1: 86.75%)
KSWIN - Data drift detectado en el ejemplo 3355 (F1: 85.84%)
KSWIN - Data drift detectado en el ejemplo 4054 (F1: 86.47%)
Resultado final (HoeffdingAdaptiveTreeClassifier): F1: 86.82%


### **Concept Drift en ExtremelyFastDecisionTreeClassifier**

Finalmente, probamos el árbol de decisiones extremadamente rápido, junto con los detectores de drift.


In [18]:
# crear, entrenar y evaluar modelo
model_extreme = tree.ExtremelyFastDecisionTreeClassifier(grace_period=120)
metric = metrics.F1()

# detectores de drift
kswin = drift.KSWIN()
eddm = drift.EDDM()
ddm = drift.DDM()

for index, row in df.iterrows():
    x = {
        'num_caracteres': row['num_caracteres'],
        'num_palabras': row['num_palabras'],
        'num_alfabeticos': row['num_alfabeticos'],
        'num_numericos': row['num_numericos'],
        'num_no_alfanum': row['num_no_alfanum'],
        'num_divisas': row['num_divisas'],
        'num_mayusculas': row['num_mayusculas'],
        'num_exclamaciones': row['num_exclamaciones'],
        'num_interrogaciones': row['num_interrogaciones'],
        'num_urls': row['num_urls']
    }

    y = row['spam'] 
    y_pred = model_extreme.predict_one(x)
    model_extreme.learn_one(x, y)
    metric.update(y, y_pred)
    
    eddm.update(int(y == y_pred))
    ddm.update(int(y == y_pred))
    kswin.update(int((x['num_mayusculas'])))
    
    if eddm.drift_detected: 
        print(f"EDDM - Model drift detectado en el ejemplo {index} ({metric})")
    if ddm.drift_detected:
        print(f"DDM - Model drift detectado en el ejemplo {index} ({metric})")
    if kswin.drift_detected:
        print(f"KSWIN - Data drift detectado en el ejemplo {index} ({metric})")

print(f'Resultado final (ExtremelyFastDecisionTreeClassifier): {metric}')

EDDM - Model drift detectado en el ejemplo 48 (F1: 14.29%)
EDDM - Model drift detectado en el ejemplo 111 (F1: 16.00%)
EDDM - Model drift detectado en el ejemplo 161 (F1: 50.00%)
EDDM - Model drift detectado en el ejemplo 216 (F1: 53.57%)
DDM - Model drift detectado en el ejemplo 414 (F1: 69.64%)
EDDM - Model drift detectado en el ejemplo 654 (F1: 79.14%)
EDDM - Model drift detectado en el ejemplo 845 (F1: 82.17%)
EDDM - Model drift detectado en el ejemplo 914 (F1: 83.10%)
KSWIN - Data drift detectado en el ejemplo 3347 (F1: 85.12%)
KSWIN - Data drift detectado en el ejemplo 3494 (F1: 85.20%)
Resultado final (ExtremelyFastDecisionTreeClassifier): F1: 85.23%


## **Agrupamiento y Detección de Anomalías**

En esta sección, utilizamos el método no supervisado de KMeans para agrupar los datos en clústeres. Posteriormente, normalizamos los datos y detectamos las anomalías en función de la distancia euclidiana de cada dato al centroide de su clúster asignado.

### Pasos:
1. Normalizar los datos usando `StandardScaler`.
2. Agrupar los datos usando `KMeans` con 5 clústeres.
3. Calcular la distancia al centroide del clúster asignado para cada dato.
4. Detectar anomalías si la distancia al centroide es mayor a un umbral definido.

In [19]:
# Importar librerías necesarias
from river import preprocessing, cluster
import math

# Crear un escalador para estandarizar los datos
scaler = preprocessing.StandardScaler()

# Inicializar KMeans con 5 clústeres
kmeans = cluster.KMeans(n_clusters=5, seed=42)

# Umbral para considerar un punto como anomalía basándonos en la distancia al centroide
threshold = 7.0

# Para almacenar el conteo de anomalías detectadas
anomalies_detected = 0

# Función para calcular la distancia euclidiana entre un punto y un centroide
def euclidean_distance(point, center):
    return math.sqrt(sum((point[feature] - center[feature]) ** 2 for feature in point))


### **Flujo de Datos para Detección de Anomalías**

En esta sección, simulamos el procesamiento en flujo continuo de datos (streaming) y escalamos las características y las agrupamos en clústeres utilizando KMeans. Detectaremos anomalías cuando la distancia al centroide sea mayor que el umbral definido.

In [20]:
for index, row in df.iterrows():
    features = {
        'num_caracteres': float(row['num_caracteres']),
        'num_palabras': float(row['num_palabras']),
        'num_alfabeticos': float(row['num_alfabeticos']),
        'num_numericos': float(row['num_numericos']),
        'num_no_alfanum': float(row['num_no_alfanum']),
        'num_divisas': float(row['num_divisas']),
        'num_mayusculas': float(row['num_mayusculas']),
        'num_exclamaciones': float(row['num_exclamaciones']),
        'num_interrogaciones': float(row['num_interrogaciones']),
        'num_urls': float(row['num_urls'])
    }

    scaler.learn_one(features)
    
    features_scaled = scaler.transform_one(features)

    cluster_id = kmeans.predict_one(features_scaled)

    centroids = kmeans.centers

    if centroids:
        centroid = centroids[cluster_id]

        distance_to_centroid = euclidean_distance(features_scaled, centroid)

        if distance_to_centroid > threshold:
            anomalies_detected += 1
            print(f"Anomalía detectada en {index} con distancia {distance_to_centroid:.2f}")

    kmeans.learn_one(features_scaled)

print(f"Total de anomalías detectadas: {anomalies_detected}")

Anomalía detectada en 155 con distancia 7.91
Anomalía detectada en 164 con distancia 8.10
Anomalía detectada en 188 con distancia 7.53
Anomalía detectada en 401 con distancia 7.98
Anomalía detectada en 445 con distancia 10.16
Anomalía detectada en 455 con distancia 7.47
Anomalía detectada en 492 con distancia 7.68
Anomalía detectada en 793 con distancia 13.31
Anomalía detectada en 838 con distancia 8.99
Anomalía detectada en 1036 con distancia 7.09
Anomalía detectada en 1085 con distancia 25.62
Anomalía detectada en 1385 con distancia 7.57
Anomalía detectada en 1463 con distancia 8.08
Anomalía detectada en 1487 con distancia 7.12
Anomalía detectada en 1546 con distancia 8.04
Anomalía detectada en 1579 con distancia 10.47
Anomalía detectada en 1586 con distancia 7.78
Anomalía detectada en 1609 con distancia 9.89
Anomalía detectada en 1659 con distancia 8.96
Anomalía detectada en 1793 con distancia 7.29
Anomalía detectada en 1827 con distancia 7.81
Anomalía detectada en 1863 con distanci

## **Tratamiento de Texto**

En esta sección, compararemos dos técnicas para la extracción de características de texto:
- **Bag of Words (BOW)**: Simplemente cuenta las ocurrencias de cada palabra en el texto.
- **TF-IDF (Term Frequency-Inverse Document Frequency)**: Pondera la frecuencia de las palabras considerando también su relevancia en el conjunto total de datos.

Ambas técnicas serán combinadas con un clasificador Naive Bayes para evaluar cuál de las dos consigue mejores resultados en la clasificación de mensajes SPAM.

### **Bag of Words con Naive Bayes**

En este apartado implementamos la técnica de Bag of Words junto con el clasificador Naive Bayes. Entrenaremos el modelo y evaluaremos su rendimiento usando la métrica **F1 Score**.

In [21]:
from river import feature_extraction, naive_bayes, compose, metrics

bow = feature_extraction.BagOfWords(lowercase=True)

model_bayes = naive_bayes.MultinomialNB()

pipeline_BowNb = compose.Pipeline(('vectorizer', bow), ('nb', model_bayes))

metric = metrics.F1()

for index, row in df.iterrows():
    text = row['text']
    y = row['spam']
    
    y_pred = pipeline_BowNb.predict_one(text)
    
    metric.update(y, y_pred)
    
    pipeline_BowNb.learn_one(text, y)

print(f"F1 Score final con Bag of Words: {metric.get()}")


F1 Score final con Bag of Words: 0.9002590673575129


### **TF-IDF con Naive Bayes**

En esta parte, utilizamos la técnica de TF-IDF junto con el clasificador Naive Bayes. Evaluaremos su rendimiento en flujo de datos de la misma manera que lo hicimos con Bag of Words.

In [22]:
tfidf = feature_extraction.TFIDF(lowercase=True, ngram_range=(1,1))
model_bayes = naive_bayes.MultinomialNB()
pipeline_TFIDFNb = compose.Pipeline(('vectorizer', tfidf), ('TFI-IDF', model_bayes))

metric = metrics.F1()

for index, row in df.iterrows():
    text = row['text']
    y = row['spam']
    
    y_pred = pipeline_TFIDFNb.predict_one(text)
    
    metric.update(y, y_pred)
    
    pipeline_TFIDFNb.learn_one(text, y)

print(f"F1 Score final con TF-IDF: {metric.get()}")

F1 Score final con TF-IDF: 0.7371090448013524


## **Ensembles (Vacío)**



## **Flujo de datos Simulado**

In [14]:
import time

try:
    for index, row in df.iterrows():
        X = {
            'text': row['text'],
            'num_caracteres': row['num_caracteres'],
            'num_palabras': row['num_palabras'],
            'num_alfabeticos': row['num_alfabeticos'],
            'num_numericos': row['num_numericos'],
            'num_no_alfanum': row['num_no_alfanum'],
            'num_divisas': row['num_divisas'],
            'num_mayusculas': row['num_mayusculas'],
            'num_exclamaciones': row['num_exclamaciones'],
            'num_interrogaciones': row['num_interrogaciones'],
            'num_urls': row['num_urls']
        }

        y = row['spam'] 
        
        print(f"Datos recibidos: {X}, Etiqueta: {y}")
        
        time.sleep(0.5)
        
except KeyboardInterrupt:
    print('Proceso detenido por el usuario.')

Datos recibidos: {'text': 'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...', 'num_caracteres': 111, 'num_palabras': 20, 'num_alfabeticos': 83, 'num_numericos': 0, 'num_no_alfanum': 28, 'num_divisas': 0, 'num_mayusculas': 3, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': 'Ok lar... Joking wif u oni...', 'num_caracteres': 29, 'num_palabras': 6, 'num_alfabeticos': 18, 'num_numericos': 0, 'num_no_alfanum': 11, 'num_divisas': 0, 'num_mayusculas': 2, 'num_exclamaciones': 0, 'num_interrogaciones': 0, 'num_urls': 0}, Etiqueta: 0
Datos recibidos: {'text': "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's", 'num_caracteres': 155, 'num_palabras': 28, 'num_alfabeticos': 97, 'num_numericos': 25, 'num_no_alfanum': 33, 'num_divisas': 0, 'num_mayusculas': 10, 'num_exclamaciones